<a href="https://colab.research.google.com/github/Yash-Kamtekar/Special-Topics-Assignment-5/blob/main/active_learning_with_label_studio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install lightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 507 kB 33.0 MB/s 
     |████████████████████████████████| 798 kB 57.7 MB/s 
     |████████████████████████████████| 151 kB 70.5 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 117 kB 71.0 MB/s 
     |████████████████████████████████| 529 kB 62.1 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144576 sha256=447978cb6bc2f4d08a42b1bb928ca6accb0afad3bb3e4163a1b7c9f9ec8b3deb
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=6fb2e0fcef96806585b9d0e171a02b3321483c664f9915ff8f84eec087a5d71d
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torchvision import datasets, io, models, ops, transforms, utils


import json
from typing import Dict, Tuple, List


def read_label_element(label_element: Dict) -> Tuple[str, str]:
    filepath = "/" + label_element["data"]["image"].split("?d=")[-1]
    label = label_element["annotations"][0]["result"][0]["value"]["rectanglelabels"][0]
    return filepath, label

def read_LabelStudio_label_file(filepath: str) -> Tuple[List[str],List[str]]:
    # read the label file
    with open(filepath, 'r') as json_file:
        data = json.load(json_file)
    filepaths, labels = zip(* [read_label_element(label_element) for label_element in data])
    return filepaths, labels

In [6]:
import os
import sys
from typing import List, Tuple, Union

import torch
import torchvision
from lightly.active_learning.scorers import ScorerClassification
from PIL import Image
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as T
from tqdm import tqdm

In [7]:
!lightly-magic token='7d9e3e942dbb8949325f955a838e79d822c3405cff6e0444' dataset_id='637491f296ba509f0cd0aa90' input_dir='/content/train' trainer.max_epochs=20

/usr/local/lib/python3.7/dist-packages/hydra/_internal/hydra.py:127: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  configure_logging=with_log_configuration,
########## Starting to train an embedding model.
/usr/local/lib/python3.7/dist-packages/lightly/cli/train_cli.py:69: UserWarning: Training a self-supervised model with a small batch size: 16! Small batch size may harm embedding quality. You can specify the batch size via the loader key-word: loader.batch_size=BSZ
  warnings.warn(msg)
Downloading: "https://storage.googleapis.com/models_boris/whattolabel-resnet18-simclr-d32-w1.0-i-085d0693.pth" to /root/.cache/torch/hub/checkpoints/whattolabel-resnet18-simclr-d32-w1.0-i-085d0693.pth
100% 42.8M/42.8M [00:06<00:00, 6.90MB/s]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/lightning.py:23: LightningDeprecationWarning

In [8]:
class TorchImageDataset(Dataset):
    def __init__(
        self,
        image_paths: List[str],
        label_names: List[str],
        transform: object = None,
        labels: List[str] = None,
    ):
        """
        :param image_paths:
            one path for each image
        :param label_names:
            an ordered list of possible labels
        :param transform:
        :param labels:
            one label for each image. Each label must be in the label_names.
        """
        if transform is None:
            transform = T.Compose(
                [
                    T.Resize((360, 117)),
                    T.ToTensor(),
                    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ]
            )
        self.transform = transform
        self.image_paths = image_paths
        self.labels = labels

        self.label_names = label_names
        self.label_name_to_index = {name: i for i, name in enumerate(label_names)}

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, int], torch.Tensor]:
        image_path = self.image_paths[index]
        image_pil = Image.open(image_path).convert("RGB")
        image_torch = self.transform(image_pil)
        if self.labels:
            return image_torch, self.label_name_to_index[self.labels[index]]
        else:
            return image_torch

    def __len__(self):
        return len(self.image_paths)


class ClassificationModel:
    def __init__(
        self, 
        num_classes: int = 8, 
        no_epochs: int = 5, 
        num_workers: int = None,
        batch_size: int = 1,
        **kwargs,
    ):
        # don't forget to initialize base class...
        self.model = torchvision.models.resnet18(
            pretrained=False, progress=True, num_classes=num_classes
        )
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(self.device)
        self.no_epochs = no_epochs
        self.model_is_trained = False
        self.num_workers = num_workers or min(8, os.cpu_count())
        self.batch_size = batch_size

    def save_on_disk(self, model_path: str = "./classifier.pth"):
        to_save = {"model": self.model.to("cpu"), "label_names": self.label_names}
        torch.save(to_save, model_path)

    def load_from_disk(self, model_path: str = "./classifier.pth"):
        saved_data = torch.load(model_path)
        self.label_names = saved_data["label_names"]
        self.model = saved_data["model"].to(self.device)
        self.model_is_trained = True

    def fit(
        self,
        image_paths: List[str],
        image_labels: List[str],
        label_names: List[str] = None,
    ):
        print("STARTING FITTING")
        if label_names is None:
            self.label_names = sorted(list(set(image_labels)))

        dataset = TorchImageDataset(
            image_paths=image_paths, label_names=self.label_names, labels=image_labels
        )
        
        dataloader = torch.utils.data.DataLoader(
            dataset, 
            batch_size=self.batch_size, 
            shuffle=True, 
            num_workers=self.num_workers,
        )
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
        self.model.train()
        pbar = tqdm(range(self.no_epochs), file=sys.stdout)
        for epoch in pbar:  # loop over the dataset multiple times
            running_loss = 0.0
            total_samples = 0
            correct = 0
            for data in dataloader:
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_samples += labels.size(0)
                correct += (predicted == labels).sum().item()
            text = f"epoch: {epoch} loss: {running_loss / total_samples:.6f} accuracy: {correct/total_samples:.3f}"
            tqdm.write(text)

        self.model_is_trained = True
        print("FINISHED FITTING")

    def predict(self, image_paths: List[str]) -> Tuple[List[str], ScorerClassification]:
        print("STARTING PREDICTION")

        if not self.model_is_trained:
            raise ValueError

        dataset = TorchImageDataset(image_paths, self.label_names)

        dataloader = DataLoader(
            dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
        )
        predictions = []
        self.model.eval()
        with torch.no_grad():
            for x in tqdm(dataloader):
                pred = self.model(x.to(self.device)).cpu()
                predictions.append(pred)

        print("PUTTING TOGETHER RETURN VALUES")

        # flatten over batches
        predictions = [i for sublist in predictions for i in sublist]
        predictions = torch.stack(predictions, dim=0)
        predictions = torch.nn.functional.softmax(predictions, dim=1)

        predicted_classes_int = torch.argmax(predictions, dim=1)
        predicted_classes_str = [self.label_names[i] for i in predicted_classes_int]
        scorer = ScorerClassification(predictions)

        print("FINISHED PREDICTION")

        return predicted_classes_str, scorer

In [17]:
import os

from lightly.active_learning.agents import ActiveLearningAgent
from lightly.active_learning.config import SelectionConfig
from lightly.api import ApiWorkflowClient
from lightly.data import LightlyDataset
from lightly.openapi_generated.swagger_client import SamplingMethod

if __name__ == "__main__":
    path_full_dataset = "/content/train" ## UPDATE THIS PATH

    # 1. Prepare the active learning agent to make sure that your credentials are correct
    lighty_webapp_token = "7d9e3e942dbb8949325f955a838e79d822c3405cff6e0444"
    lighty_webapp_dataset_id = "637491f296ba509f0cd0aa90"
    api_workflow_client = ApiWorkflowClient(token=lighty_webapp_token, dataset_id=lighty_webapp_dataset_id)
    al_agent = ActiveLearningAgent(api_workflow_client=api_workflow_client)

    # 2. read the label file
    filepaths, labels = read_LabelStudio_label_file("/content/jsonformat.json") ##UPDATE THIS PATH

    # 3. Define the image classification model
    classifier = ClassificationModel(no_epochs=20)

    # 4. Fit the classifier on the labeled images
    classifier.fit(image_paths=filepaths, image_labels=labels)

    # 5. Predict with the classifier on the complete dataset
    image_filenames_full_dataset = LightlyDataset(path_full_dataset).get_filenames()
    image_paths_full_dataset = [os.path.join(path_full_dataset, filename) for filename in image_filenames_full_dataset]
    predicted_classes_str, scorer = classifier.predict(image_paths=image_paths_full_dataset)

    # 6. Use the active learning sampler "CORAL" via the Lightly API to sample until we have 45 samples
    sampler_config = SelectionConfig(method=SamplingMethod.CORAL, n_samples=45, name="datasets")
    al_agent.query(sampler_config, al_scorer=scorer)


STARTING FITTING
  0%|          | 0/20 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4d6d3c93b0>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f4d6d3c93b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    self._shutdown_workers()    assert self._parent_pid == os.getpid(), 'can only test a child process'

  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():AssertionError
:   File "/usr/lib/python3.7/multiprocessing/process.p

epoch: 0 loss: 1.967100 accuracy: 0.333
epoch: 1 loss: 2.685572 accuracy: 0.056
epoch: 2 loss: 2.571573 accuracy: 0.167
epoch: 3 loss: 2.138093 accuracy: 0.111
epoch: 4 loss: 2.041456 accuracy: 0.222
epoch: 5 loss: 2.309763 accuracy: 0.278
epoch: 6 loss: 1.261855 accuracy: 0.500
epoch: 7 loss: 1.871713 accuracy: 0.167
epoch: 8 loss: 1.125784 accuracy: 0.444
epoch: 9 loss: 0.822802 accuracy: 0.778
epoch: 10 loss: 0.613614 accuracy: 0.944
epoch: 11 loss: 0.459094 accuracy: 1.000
epoch: 12 loss: 0.309658 accuracy: 1.000
epoch: 13 loss: 0.272038 accuracy: 1.000
epoch: 14 loss: 0.153584 accuracy: 1.000
epoch: 15 loss: 0.102112 accuracy: 1.000
epoch: 16 loss: 0.084394 accuracy: 1.000
epoch: 17 loss: 0.067493 accuracy: 1.000
epoch: 18 loss: 0.057572 accuracy: 1.000
epoch: 19 loss: 0.050049 accuracy: 1.000
100%|██████████| 20/20 [00:09<00:00,  2.09it/s]
FINISHED FITTING
STARTING PREDICTION


100%|██████████| 299/299 [00:02<00:00, 135.18it/s]


PUTTING TOGETHER RETURN VALUES
FINISHED PREDICTION
